In [1]:
import os
from os.path import join
import pickle
import numpy as np
import pandas as pd
from Bio import SeqIO
from collections import Counter
from scipy.stats import binom_test
from statsmodels.stats.multitest import fdrcorrection
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


Bad key "text.kerning_factor" on line 4 in
/Users/sudeshnaghosh/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
"""
Imports and merges NanoARG Full Table and assembly information (length and read-count).
Calculates quantities and relative abundances of scaffolds with ARGs, MRGs, MGEs and 
pathogen scaffols. 

"""

'\nImports and merges NanoARG Full Table and assembly information (length and read-count).\nCalculates quantities and relative abundances of scaffolds with ARGs, MRGs, MGEs and \npathogen scaffols. \n\n'

In [3]:
def remove_overlap_not_ARG_strand_selection(df, 
                                            max_overlap=250, 
                                            verbose=False, 
                                            strand_selection='no'):
    """ 
    Rearrange open reading frames according to position.
    Find open reading frames identified as ARGs that overlap with other 
    (MGEs, MRGs, Functional Genes) open reading frames and 
    remove them. 
    It is necessary to run this before many of the other functions.
    For example, the same region identified as ARG and MGE can result in 
    misidentification of contigs with both ARG and MGE. Overlapping 
    open reading frame identifications also make it more difficult to analyze 
    contiguous regions.
    
    Example
    -------
          
    s1 ----ARGs------- e1 
         s2 --------- e2 removed if overlap > max_overlap
                  s3-------------e3 keep if overlap < max_overlap
                           s4 ------ e4 
    
    Parameters
    ----------
    df : pd.DataFrame
        data frame with NanoARG table
    max_overlap : int
        maximum allowed overlap between consequitive orfs
    
    strand_selection : string
        choose 'no','yes'
        
        default run with 'no', preferred for subsequent data analysis
        ignore +/- strand
        (11) scaffold_10 4879-6615 ARGs
           (12) 5947-6558  removing Functional Genes! overlap 611
        (27) scaffold_10 44937-45410 ARGs
        (30) scaffold_10 47395-49008 ARGs
           (31) 47395-48969  removing MRGs! overlap 1574
           (32) 47443-48996  removing Functional Genes! overlap 1553
        (36) scaffold_10 52079-52978 ARGs
        (37) scaffold_10 56260-57171 ARGs
           (38) 56260-57168  removing Functional Genes! overlap 908
        (42) scaffold_10002 34-753 ARGs
           (43) 40-768  removing Functional Genes! overlap 713
           
        run with 'yes'
        only remove if orfs have same 5'-> 3' direction
        
        (11) scaffold_10 4879-6615 ARGs
           (12) 5947-6558 strand + + removing Functional Genes! overlap 611
        (27) scaffold_10 44937-45410 ARGs
        (30) scaffold_10 47395-49008 ARGs
           (31) 47395-48969 strand + - overlap 1574
           (32) 47443-48996 strand + - overlap 1553
        (36) scaffold_10 52079-52978 ARGs
        (37) scaffold_10 56260-57171 ARGs
           (38) 56260-57168 strand + - overlap 908
        (42) scaffold_10002 34-753 ARGs
           (43) 40-768 strand + + removing Functional Genes! overlap 713
        

    verbose : bool
        print debug info
        
    Returns
    -------
    Dataframe with entries removed
    
    
   
    """
    if verbose: 
        print("sorting...")
        
    group_column = 2

    D_sorted = df.groupby(df.index).apply(
        lambda x: x.sort_values('start', ascending=True))
    
    Ddrop = D_sorted.droplevel(level=0).copy()

    if verbose: 
        print("searching")

    for i in range(len(Ddrop)-1):
        if Ddrop.group.iloc[i] == 'ARGs':
            scaffold = Ddrop.index[i]

            if verbose: 
                print("({}) {} {}-{} {}".format(
                    i, scaffold, Ddrop.start.iloc[i], Ddrop.end.iloc[i], Ddrop.group.iloc[i]))

            j = i + 1
            while (j<len(Ddrop) and (Ddrop.start.iloc[j] <= Ddrop.end.iloc[i])
                   and (Ddrop.index[j] == scaffold)):

                if verbose:
                    print("   ({}) {}-{} ".format(
                        j, Ddrop.start.iloc[j], Ddrop.end.iloc[j]), end='')

                overlap = np.min([Ddrop.end.iloc[j], Ddrop.end.iloc[i]]) - Ddrop.start.iloc[j]
                if overlap >= max_overlap:
                    if Ddrop.group.iloc[j] != 'ARGs':
                        if strand_selection == 'yes':
                            if verbose:
                                print("strand {} {}".format(
                                    Ddrop.strand.iloc[i], Ddrop.strand.iloc[j]), end = '')
                               
                            if Ddrop.strand.iloc[j] == Ddrop.strand.iloc[i]:
                                if verbose:
                                    print(" removing {}!".format(Ddrop.group.iloc[j]), end='')
                                Ddrop.iat[j, group_column] = np.nan
                        elif strand_selection == 'no':
                            if verbose:
                                print(" removing {}!".format(Ddrop.group.iloc[j]), end='')
                            Ddrop.iat[j, group_column] = np.nan
                else:

                    if verbose:
                        print("keeping {}!".format(Ddrop.group.iloc[j]), end='')

                if verbose:
                    print(" overlap {}".format(overlap))

                j = j + 1      
                
    for i in range(1, len(Ddrop)):
        if Ddrop.group.iloc[i] == 'ARGs':
            scaffold = Ddrop.index[i]

            if verbose: 
                print("({}) {} {}-{} {}".format(
                    i, scaffold, Ddrop.start.iloc[i], Ddrop.end.iloc[i], Ddrop.group.iloc[i]))

     
            j = i - 1
            while (j>=0 and (Ddrop.start.iloc[i] <= Ddrop.end.iloc[j])
                   and (Ddrop.index[j] == scaffold)):

                if verbose:
                    print("   ({}) {}-{} ".format(
                        j, Ddrop.start.iloc[j], Ddrop.end.iloc[j]), end='')

                overlap = np.min([Ddrop.end.iloc[j], Ddrop.end.iloc[i]]) - Ddrop.start.iloc[i]
                if overlap >= max_overlap:
                    if Ddrop.group.iloc[j] != 'ARGs':
                        if strand_selection == 'yes':
                            if verbose:
                                print("strand {} {}".format(
                                    Ddrop.strand.iloc[i], Ddrop.strand.iloc[j]), end='')
                            if Ddrop.strand.iloc[j] == Ddrop.strand.iloc[i]:
                                if verbose:
                                    print(" removing {}!".format(Ddrop.group.iloc[j]), end='')
                                Ddrop.iat[j, group_column] = np.nan
                        elif strand_selection == 'no':
                            if verbose:
                                print(" removing {}!".format(Ddrop.group.iloc[j]), end='')
                            Ddrop.iat[j, group_column] = np.nan
                    
                else:

                    if verbose:
                        print("keeping {}!".format(Ddrop.group.iloc[j]), end='')

                if verbose:
                    print(" overlap {}".format(overlap))

                j = j - 1
                
    df_remove_overlap = Ddrop.dropna(subset=['group']) 
    
    return df_remove_overlap       

In [4]:
"""
Necessary to run 'remove_overlap_not_ARG_strand_selection' before. 
Calculates quantities and relative abundances of scaffolds with ARGs,
MGEs and scaffolds identified as pathogens.
"""

def count_scaffolds_with_MGEs_and_ARGs_normalized(df, min_score=40000, 
                                                  min_length = 500):
    """
    Count
        ARG_counts
        scaffold_ARGs
        scaffold_ARGs_normalized
        scaffold_MGEs
        scaffold_MGEs_with_ARGs
        scaffold_MGEs_with_ARGs_normalized
        %_scaffold_MGEs_with_ARGs/scaffold_ARGs
        scaffold_pathogens
        scaffold_pathogens_with_args
        scaffold_pathogens_with_mges
        scaffold_pathogens_with_args_mges
        scaffold_has_pathogens_ARGs_MGEs_normalized
    
    
    Parameters
    ----------
    df : pd.dataframe
        dataframe with scaffold, ARGs and MGEs data
    min_score : int
        taxa_centrifuge_score cutoff for taxonomy classification
    min_length : int
        minimum length of scaffold to include in analysis
    
    Returns
    -------
    dict:
        dictionary with ratio and counts
        
        
    
    
    """
    
    df_length_filtered = df[df['length'] > min_length]
    
    scaffold_has_args_normalized = 0
    scaffold_has_args=0
    old_index_arg = 0
    for i in range(len(df_length_filtered)):
        if df_length_filtered.group.iloc[i] == 'ARGs':
            if df_length_filtered.index[i] != old_index_arg:
                scaffold_has_args += 1
                scaffold_has_args_normalized  += (
                    df_length_filtered.normalized_coverage.iloc[i])
                old_index_arg = df_length_filtered.index[i]
    
    
    scaffold_has_mges = df_length_filtered.groupby(level=0).apply(
        lambda x: (x['group']=='MGEs').any()
    ).sum()
    
    scaffold_has_mges_with_args=0
    scaffold_has_mges_with_args_norm=0
    same_index=0
    for scaffold, data in df_length_filtered.groupby(level=0):
        groups = np.array(data.group)
        normalized_coverage = np.array(data.normalized_coverage)
        indices = np.array(data.index)
        for i in range(len(groups)):
            if (groups[i] == 'ARGs'):
                for j in range(len(groups)):
                    if (groups[j] == 'MGEs'):
                        if indices[j] != same_index:
                            scaffold_has_mges_with_args += 1
                            scaffold_has_mges_with_args_norm += normalized_coverage[j]
                            same_index = indices[j]
        
    
    scaffold_has_pathogens = df_length_filtered.groupby(level=0).apply(
        lambda x: 
            (x['is_pathogen']==1).any() 
            and (x['taxa_centrifuge_score']>min_score).any()
    ).sum()
    
    scaffold_has_pathogens_args = df_length_filtered.groupby(level=0).apply(
        lambda x: 
            (x['is_pathogen']==1).any() 
            and (x['taxa_centrifuge_score']>min_score).any() 
            and (x['group']=='ARGs').any()
    ).sum()
    
    scaffold_has_pathogens_mges = df_length_filtered.groupby(level=0).apply(
        lambda x: 
            (x['is_pathogen']==1).any() 
            and (x['taxa_centrifuge_score']>min_score).any()
            and (x['group']=='MGEs').any() 
    ).sum()
    
    df_length_taxa_filtered = df_length_filtered[df_length_filtered['taxa_centrifuge_score'] > min_score]
    scaffold_has_pathogens_args_mges=0
    scaffold_has_pathogens_args_mges_norm=0
    same_index_pathogen=0
    for scaffold, data in df_length_taxa_filtered.groupby(level=0):
        groups = np.array(data.group)
        normalized_coverage = np.array(data.normalized_coverage)
        indices = np.array(data.index)
        pathogens = np.array(data.is_pathogen)
        for i in range(len(groups)):
            if (pathogens[i] == 1):
                for j in range(len(groups)):
                    if (groups[j] == 'ARGs'):
                        for k in range(len(groups)):
                            if (groups[k] == 'MGEs'):
                                if indices[k] != same_index_pathogen:
                                    scaffold_has_pathogens_args_mges += 1
                                    scaffold_has_pathogens_args_mges_norm += normalized_coverage[k]
                                    same_index_pathogen = indices[k]
    
    ARG_counts = len(df_length_filtered[df_length_filtered['group'] == 'ARGs'])
    
    
    return {
        'ARG_counts' : ARG_counts,
        'scaffold_ARGs': scaffold_has_args,
        'scaffold_ARGs_normalized' : scaffold_has_args_normalized,
        'scaffold_MGEs': scaffold_has_mges,
        'scaffold_MGEs_with_ARGs': scaffold_has_mges_with_args,
        'scaffold_MGEs_with_ARGs_normalized' :scaffold_has_mges_with_args_norm,
        '%_scaffold_MGEs_with_ARGs/scaffold_ARGs': (
            scaffold_has_mges_with_args / scaffold_has_args * 100
        ),
        'scaffold_pathogens' : scaffold_has_pathogens,
        'scaffold_pathogens_with_ARGs' : scaffold_has_pathogens_args,
        'scaffold_pathogens_with_MGEs' : scaffold_has_pathogens_mges,
        'scaffold_pathogens_with_ARGs_MGEs' : scaffold_has_pathogens_args_mges,
        'scaffold_has_pathogens_ARGs_MGEs_normalized' : scaffold_has_pathogens_args_mges_norm
    }

In [5]:
"""
Returns quantity and relative abundance of desired ARG subtype scaffold. 
"""

def ARG_subtype_count(df, min_bitscore=50, 
                      min_id_arg=25, min_coverage_arg=0.4, arg_name = 'sul1'):
    
    """
    Count
        arg_subtype_count = count # of ARG subtype occurance
        scaffold_arg_subtype = count # of scaffolds with ARG_subtype
        scaffold_has_arg_subtype_normalized = scaffolds with ARG_subtype in RPKM
    
    
    Parameters
    ----------
    df : pd.dataframe
        dataframe with scaffold, ARGs and MGEs data
    min_bitscore : int
        minimum bitscore, default is 50
    min_id_arg : int
        minimum identity of ARG, default is 25    
    min_coverage_arg : int
        minimum coverage of ARG, default is 0.4 in NanoARG
    arg_name : name of ARG subtype to be counted
    
    Returns
    -------
    dict:
        dictionary with counts   
    """
    
    
    
    df_bitscore_adjusted = df[df['bitscore'] > min_bitscore]
    
    df_bit_arg_adjusted = df_bitscore_adjusted.drop(df_bitscore_adjusted[ 
        (df_bitscore_adjusted['group'] == 'ARGs') 
        & ((df_bitscore_adjusted['identity'] < min_id_arg)
            | (df_bitscore_adjusted['coverage'] < min_coverage_arg))].index)
    
    
    arg_subtype_count = len(df_bit_arg_adjusted[df_bit_arg_adjusted['gene_name'] == arg_name])
    
    scaffold_has_arg_subtype = df_bit_arg_adjusted.groupby(level=0).apply(
        lambda x: (x['gene_name']==arg_name).any()
    ).sum()
    
    
    scaffold_has_arg_subtype_normalized = 0
    old_index = 0
    for i in range(len(df_bit_arg_adjusted)):
        if df_bit_arg_adjusted.gene_name.iloc[i] == arg_name:
            if df_bit_arg_adjusted.index[i] != old_index:              
                scaffold_has_arg_subtype_normalized  += (
                    df_bit_arg_adjusted.normalized_coverage.iloc[i])
                old_index = df_bit_arg_adjusted.index[i]
                
    return {
        'arg_subtype_count' : arg_subtype_count,
        'scaffold_arg_subtype' : scaffold_has_arg_subtype,
        'scaffold_has_arg_subtype_normalized' : scaffold_has_arg_subtype_normalized
    }

In [6]:
def count_scaffolds(df, min_length=500):
    """    
    Count number of scaffolds longer than min_length 
    
    Parameters
    ----------
    df : pd.dataframe
        dataframe with scaffold length and read-count
    min_length : int
        minimum length of scaffold to include in analysis
    
    Returns
    -------
    dict:
        dictionary with counts
           
    """
    
    df_length_filtered = df[df['length'] > min_length]
    scaffold_counts = len(df_length_filtered.groupby(level=0))
    
    return {
        '# scaffolds' : scaffold_counts,
    }

In [46]:
"""
It is necessary to run 'remove_overlap_not_ARG_strand_selection' before.
Finds likely association between ARG subtype and MGE.

"""

def ARG_MGE_association(df, min_bitscore=50, min_id_arg=25, min_id_mge=25,
                         f=0.01, max_fdr=0.05, min_coverage_arg=0.4,
                       min_coverage_mge=0.4, min_length=500):

    """
    
    Parameters
    ----------
    df : pd.dataframe
        dataframe with scaffold, ARGs and MGEs data
    min_bitscore : int
        minimum bitscore, default is 50
    min_id_arg : int
        minimum identity of ARG, default is 25    
    min_id_mge : int
        minimum identity of MGE, default is 25
    min_coverage_arg : int
        minimum coverage of ARG, default is 0.4 in NanoARG
    min_coverage_mge : int
        minimum coverage of MGE, default is 0.4 in NanoARG
    f : float
        ratio of scaffolds with MGEs and total number of scaffolds
        set f = 0 to bypass binom test with FDR (Benjamini/Hochberg) correction.
        Binom test checks if fraction of ARG subtype scaffold with MGEs 
        exceeds fraction of scaffolds with MGEs in the sample, in other words,
        if ARG subtype is associated with an MGE more often than expected 
    max_fdr : float
        maximum FDR value to include in output 
    min_length : int
        minimum length of scaffold to include in analysis
        
    Returns
    -------
    fraction_genes_with_mge_test_count: pd.DataFrame()
        Dataframe with ratio and counts
        fdr : corrected p-value
        fraction_genes_with_mge : ARG subtype scaffold with MGE / ARG subtype scaffold
        count_genes : count of ARG subtype
        
    
    """
    df_length_filtered = df[df['length'] > min_length]
    
    df_bitscore_adjusted = df_length_filtered[df_length_filtered['bitscore'] > min_bitscore]

    df_bit_arg_adjusted = df_bitscore_adjusted.drop(df_bitscore_adjusted[ 
        (df_bitscore_adjusted['group'] == 'ARGs') 
        & ((df_bitscore_adjusted['identity'] < min_id_arg)
            | (df_bitscore_adjusted['coverage'] < min_coverage_arg))].index)

    df_adjusted = df_bit_arg_adjusted.drop(df_bit_arg_adjusted[ 
        (df_bit_arg_adjusted['group'] == 'MGEs') 
        & ((df_bit_arg_adjusted['identity'] < min_id_mge)
            | (df_bit_arg_adjusted['coverage'] < min_coverage_mge))].index)
    
    D = df_adjusted[df_adjusted.group.isin(['ARGs','MGEs'])]
    D_args = df_adjusted[df_adjusted.group.isin(['ARGs'])]
    count_genes = Counter(D_args['gene_name'])
    scaffolds = D_args.index.unique()

    count_genes_with_mge = Counter()
    for i, scaffold in enumerate(scaffolds):
        print("{:.2f}%".format((i+1)/len(scaffolds)*100.), end='\r')

        in_scaffold = D[D.index==scaffold]
        if 'MGEs' in D[D.index==scaffold]['group'].values:

            for gene in count_genes:
                if gene in in_scaffold['gene_name'].values:
                    count_genes_with_mge[gene]+=1
    print("           ", end="\r")

    count_genes_once_per_scaffold = Counter()
    for i, scaffold in enumerate(scaffolds):
        print("{:.2f}%".format((i+1)/len(scaffolds)*100.), end='\r')
        in_scaffold = D[D.index==scaffold]
        for gene in count_genes:
            if gene in in_scaffold['gene_name'].values:
                count_genes_once_per_scaffold[gene]+=1
    
    

    fraction_genes_with_mge = {}
    for gene in count_genes_with_mge:
        fraction_genes_with_mge[gene]=count_genes_with_mge[gene]/count_genes_once_per_scaffold[gene]
    
    fraction_genes_with_mge_df = pd.DataFrame.from_dict(fraction_genes_with_mge, orient='index')
    
    fraction_genes_with_mge_df = fraction_genes_with_mge_df.rename(columns={0: "fraction_genes_with_mge"})
    
    count_genes_df = pd.DataFrame.from_dict(count_genes, orient='index')
    count_genes_df = count_genes_df.rename(columns={0: "count_genes"})

    test_enriched = {}
    for gene, count in count_genes_with_mge.items(): 
        test_enriched[gene] = binom_test(count, n=count_genes_once_per_scaffold[gene], p=f, alternative='greater')
    
    test_enriched = pd.Series(test_enriched)
    
    rejected, pvalue_corrected = (
        fdrcorrection(test_enriched, alpha=max_fdr, method='indep', is_sorted=False))

    test_enriched_fdr = {
        'rejected': rejected.tolist(), 
        'fdr': pvalue_corrected.tolist(), 
        'gene_name': test_enriched.index.tolist()
    }
    test_enriched_fdr_df = pd.DataFrame(test_enriched_fdr)
    test_enriched_fdr_df.index = test_enriched_fdr_df['gene_name']

    fraction_genes_with_mge_test = (
        pd.concat([test_enriched_fdr_df, fraction_genes_with_mge_df], axis=1, join='inner'))
    
    fraction_genes_with_mge_test = (
        fraction_genes_with_mge_test.sort_values(by=['fraction_genes_with_mge'], ascending=False))
    
    fraction_genes_with_mge_test_count = (
        pd.concat([fraction_genes_with_mge_test, count_genes_df], axis=1, join='inner'))
    
    fraction_genes_with_mge_test_count = (
        fraction_genes_with_mge_test_count[fraction_genes_with_mge_test_count['rejected']])
    
    fraction_genes_with_mge_test_count.drop(['rejected','gene_name'], axis=1, inplace=True)
    return fraction_genes_with_mge_test_count

In [47]:
"""
It is necessary to run 'remove_overlap_not_ARG_strand_selection' before.
Returns stretches of scaffolds with ARGs and MGEs.

"""

def neighbor_stretch(df, min_ARG_count=1, max_distance=5000, 
                     number_neighboring_orf=4, 
                     sliding_window_step_increase=4, min_bitscore=50, 
                     min_id_arg=25, min_id_mge=25, min_coverage_arg=0.4, 
                     min_coverage_mge=0.4, verbose=False):
    
    """
    Search for MGE and specified number of ARGs in a desired
    stretch of scaffold window. This window slides forward at a 
    specified step.
    Reports a dataframe of neighboring orfs with MGEs and ARGs.
    
    Exp. number_neighboring_orf=4, sliding_window_step_increase=1
    max_distance = maximum distance between s and e
    min_ARG_count=1
    
    s1-ARG--->  <-----   ------> --MGE-->e1    report
              s2<-----   ------> --MGE--> <------e2    no ARG, do not report  
              
    
    
    Parameters
    ----------
    df : pd.dataframe
        dataframe with scaffold, ARGs, MGEs and start end 
        gene data
    
    max_distance : int
        maximum length of scaffold segment with 
        neighboring MGEs and ARGs to explore
        
    min_ARG_count : int
        minimum number of ARGs in the scaffold segment with 
        neighboring MGEs and ARGs
        
    number_neighboring_orf : int
        maximum number of neighboring orfs to consider
        
    sliding_window_step_increase : int
        int between 1 and number_neighboring_orf
    
    min_bitscore : int
        minimum bitscore, default is 50
        
    min_id_arg : int
        minimum aa identity of ARG, default is 25
        
    min_id_mge : int
        minimum aa identity of MGE, default is 25
        
    min_coverage_arg : int
        minimum coverage of ARG, default is 0.4 in NanoARG
        
    min_coverage_mge : int
        minimum coverage of MGE, default is 0.4 in NanoARG
        
    verbose : bool
        print progress and debug information
    
    Returns
    -------
    neighbor_genes : dataframe
        dataframe with neighboring ARGs and MGEs
    
    """
    neighbor_genes = {}
    
    output_columns = ['gene_id', 'gene_name', 'group', 'category', 'start', 'end', 'strand',
       'identity', 'bitscore', 'evalue', 'NCBI_taxa_id',
       'taxa_centrifuge_score', 'species', 'coverage', 'is_pathogen']
    
    for column in output_columns:
        neighbor_genes[column] = []
        
    
    neighbor_genes['read'] = []

    df_bitscore_adjusted = df[df['bitscore'] > min_bitscore]
    
    df_bit_arg_adjusted = df_bitscore_adjusted.drop(df_bitscore_adjusted[ 
        (df_bitscore_adjusted['group'] == 'ARGs') 
        & ((df_bitscore_adjusted['identity'] < min_id_arg)
            | (df_bitscore_adjusted['coverage'] < min_coverage_arg))].index)

    df_adjusted = df_bit_arg_adjusted.drop(df_bit_arg_adjusted[ 
        (df_bit_arg_adjusted['group'] == 'MGEs') 
        & ((df_bit_arg_adjusted['identity'] < min_id_mge)
            | (df_bit_arg_adjusted['coverage'] < min_coverage_mge))].index)
    
    
    if verbose:
        print("\ndf1:", df_bitscore_adjusted)
        print("\ndf2:", df_bit_arg_adjusted)
        print("\ndf3:", df_adjusted)
    
    
    for scaffold, data in df_adjusted.groupby(level=0):
        starts = np.array(data.start)
        ends = np.array(data.end)
        groups = np.array(data.group)
        
        if verbose:
            print("-----------------")
            print("\nscaffold:", scaffold)
            print("\nstarts:", starts)
            print("\nends:", ends)
            print("\ngroups:", groups)
            print("-----------------")
            print(len(groups))
        
        for i in range(0, len(groups)-number_neighboring_orf+1, sliding_window_step_increase):
            max_neighbor_orfs = min(number_neighboring_orf, len(groups) - i)
            count = Counter(groups[i:i+max_neighbor_orfs] == 'ARGs')
            
            
            if count[True] >= min_ARG_count and (groups[i:i+max_neighbor_orfs] == 'MGEs').any():
            
                if (ends[i+max_neighbor_orfs-1] - starts[i]) < max_distance:
                                
                        neighbor_genes['read'] += [
                            data.index.to_list()[i + d] for d in range(max_neighbor_orfs)
                        ]
                        
                        for column in output_columns:
                            neighbor_genes[column] += [
                                data[column].to_list()[i+d] for d in range(max_neighbor_orfs)
                            ]
    neighbor_genes = pd.DataFrame(neighbor_genes)
    neighbor_genes.index = neighbor_genes['read']
    neighbor_genes = neighbor_genes.drop(['read'], axis =1)
    
    return neighbor_genes

In [9]:
"""
INPUT REQUIRED. IMPORTING NANOARG DATA.
------------------------------------------------------

Import multiple NanoARG Full Tables (.tsv) and save in a dictionary.

Input required:
1. Path to the directory with NanoARG full tables

Output:
1. Disctionary called all_full_tables, each sample is saved 
    in a different dataframe with sample name.
 
"""



"""
***provide PATH to the directory with files*********

"""
path = '/Users/sudeshnaghosh/Dropbox/VT/test2/'


#------------------------------------------------


DATA_DIR = path

all_full_tables = {}

for file in os.listdir(DATA_DIR):
    df_name = file.split('.')[0]
    # printing sample name indicates progress
    print(df_name)
    all_full_tables[df_name] = pd.read_csv(DATA_DIR + file, sep=',', header=0, index_col=0)
    
#del all_full_tables[""]

HCL0_W2
HCL0_W3
Before_GAC_1
After_GAC_1


In [11]:
"""
INPUT REQUIRED. IMPORTING ASSEMBLY INFORMATION.
------------------------------------------------------

Extract length and read_count information about each scaffold 
in a sample and store as dataframe. Store dataframes in a dictionary.


Input:

Input file format, from IDBA-UD

>scaffold_0 length_285165 read_count_39166
CGGGCCCTACCGTAGCAGCCGCTACGGTAGGGCCCGTGTCCAGT......
>scaffold_1 length_297291 read_count_43258
GAGAGCGTCATCGATCATCGCGACGCAGGTTAACAGCGATTGCCGATGTTTACAACC........

Output:
    
Dictionary called assembly


"""

"""
***provide PATH to the directory with IDBA-UD files**********

"""

path = '/Users/sudeshnaghosh/Dropbox/VT/assembly_test'
#---------------------------------------------------


DATA_DIR = path

assembly = {}


for file in os.listdir(DATA_DIR):
    if file.endswith(".fa"):
        file_path = join(DATA_DIR, file)
        print(file_path)
        df_name = file.split('.')[0]
        scaffold = []
        length = []
        read_count = []
        
        for record in SeqIO.parse(file_path, "fasta"):
            scaffold.append(record.description.split()[0])
            length.append(record.description.split()[1].split('_')[1])
            read_count.append(record.description.rsplit()[2].split('_')[2])
            
            
        assembly[df_name] = pd.DataFrame({
            'read':scaffold,
            'length':length,
            'read_count':read_count
        })
        assembly[df_name].set_index('read', inplace=True)

/Users/sudeshnaghosh/Dropbox/VT/assembly_test/After_GAC_1.fa
/Users/sudeshnaghosh/Dropbox/VT/assembly_test/Before_GAC_1.fa
/Users/sudeshnaghosh/Dropbox/VT/assembly_test/HCL0_W2.fa
/Users/sudeshnaghosh/Dropbox/VT/assembly_test/HCL0_W3.fa


In [13]:
"""IMPORTING COVERAGE FILES"""

"""
data format (scaffold,coverage)



genome,12.541209956374683
scaffold_0,23.00672351625
scaffold_1,94.66073208748007
scaffold_10,22.6907473287
scaffold_100,17.560885991499994
scaffold_1000,23.2361095814
scaffold_10000,7.281462299999999
scaffold_100000,5.385235400000001
scaffold_100001,2.77849415
scaffold_100002,3.1011257399999996
scaffold_100003,4.524878


"""

path = '/Users/sudeshnaghosh/Dropbox/coverage_test/'


#------------------------------------------------


DATA_DIR = path

coverage = {}

for file in os.listdir(DATA_DIR):
    df_name = file.split('.')[0].rsplit('_',1)[0]
    # printing sample name indicates progress
    print(df_name)
    coverage[df_name] = pd.read_csv(DATA_DIR + file, sep=',', header=None, index_col=0)
    
#del all_full_tables[""]

Before_GAC_1
After_GAC_1
HCL0_W3
HCL0_W2


In [34]:
assembly['HCL0_W3'].head()

,length,read_count
read,,
scaffold_0,491867,60073
scaffold_1,412542,44554
scaffold_2,224903,21783
scaffold_3,195468,291957
scaffold_4,319504,57709


In [16]:
#coverage['LCM5_W3'].iloc[0,0]

In [17]:
for name, keys in coverage.items():    
    df_name = name
    # printing sample name indicates progress
    print(df_name)
    coverage[df_name] = coverage[df_name].astype(float)
    coverage[df_name]['all_scaffold'] = coverage[df_name].iloc[0,0]

Before_GAC_1
After_GAC_1
HCL0_W3
HCL0_W2


In [11]:
# """
# For each scaffold, calculate rpm and rpkm 
# -----------------------------------------


# For each scaffold, calculate rpm and rpkm and add to the disctionary with 
# length and read_count.

# normalized_read_count (rpm) = 
# read_count of scaffold / (total # of read_counts in a sample / 1,000,000)
# normalized_read_count_per_kb (rpkm) = 
# normalized_read_count (rpm) / length of scaffold in kb
# normalized_length (bases per million) =
# (length of scaffold / (length summation of scaffolds / 1,000,000))

# read_count_per_length (or 'read_count_per_base') = 
# (read_count of scaffold / (total # of read_counts in a sample / 1,000,000))
# /
# (length of scaffold / (length summation of scaffolds / 1,000,000))

# The updated dataframes are stored in the dictionary called assembly.


# """


# for name, keys in assembly.items():    
#     df_name = name
#     # printing sample name indicates progress
#     print(df_name)
#     SUM = ((assembly[df_name]['read_count']).astype(float)).sum()
#     assembly[df_name] = assembly[df_name].astype(float)
#     assembly[df_name]['normalized_read_count'] = (
#         (assembly[df_name]['read_count'] * 1000000) / SUM)
#     assembly[df_name]['normalized_read_count_per_kb'] = (
#         (assembly[df_name]['normalized_read_count'] * 1000)/
#         assembly[df_name]['length'])
#     assembly[df_name]['length_sum'] = assembly[df_name]['length'].sum()
#     assembly[df_name]['normalized_length'] = (
#         (assembly[df_name]['length'] * 1000000) / assembly[df_name]['length_sum'])
#     assembly[df_name]['read_count_per_length'] = (
#         assembly[df_name]['normalized_read_count']/
#         assembly[df_name]['normalized_length'])

In [19]:
assembly['After_GAC_1'].head()

,length,read_count
read,,
scaffold_0,709020,97243
scaffold_1,551495,312878
scaffold_2,954535,669486
scaffold_3,303743,175890
scaffold_4,302455,32823


In [20]:
"""
COMBINING ASSEMBLY INFORMATION WITH COVERAGE
--------------------------------------------

Add scaffold length, read_count and coverage.
Store dataframes in a dictionary called assembly_coverage.

"""

assembly_coverage = {}

for name, keys in assembly.items():    
    df_name = name
    # printing sample name indicates progress
    print(df_name)
    assembly_coverage[df_name] = (
        pd.concat([assembly[df_name], coverage[df_name]], 
                  axis=1, sort=False, join='inner'))

After_GAC_1
Before_GAC_1
HCL0_W2
HCL0_W3


In [21]:
assembly_coverage['HCL0_W3'].head()

,length,read_count,1,all_scaffold
scaffold_0,491867,60073,20.876421,22.253067
scaffold_1,412542,44554,18.579756,22.253067
scaffold_2,224903,21783,16.761661,22.253067
scaffold_3,195468,291957,250.051349,22.253067
scaffold_4,319504,57709,32.088281,22.253067


In [23]:
for name, keys in assembly_coverage.items():    
    df_name = name
    # printing sample name indicates progress
    print(df_name)
    assembly_coverage[df_name] = assembly_coverage[df_name].astype(float)
    assembly_coverage[df_name]['sum_assembled_read'] = assembly_coverage[df_name]['read_count'].sum()
    assembly_coverage[df_name]['length_sum'] = assembly_coverage[df_name]['length'].sum()
    assembly_coverage[df_name]['coverage_sum'] = assembly_coverage[df_name][1].sum()
    assembly_coverage[df_name]['normalized_coverage'] = (
        (assembly_coverage[df_name][1]/
        assembly_coverage[df_name]['coverage_sum']))

After_GAC_1
Before_GAC_1
HCL0_W2
HCL0_W3


In [24]:
assembly_coverage['HCL0_W3'].head()

,length,read_count,1,all_scaffold,sum_assembled_read,length_sum,coverage_sum,normalized_coverage
scaffold_0,491867.0,60073.0,20.876421,22.253067,25521993.0,235795745.0,2.716443e+06,0.000008
scaffold_1,412542.0,44554.0,18.579756,22.253067,25521993.0,235795745.0,2.716443e+06,0.000007
scaffold_2,224903.0,21783.0,16.761661,22.253067,25521993.0,235795745.0,2.716443e+06,0.000006
scaffold_3,195468.0,291957.0,250.051349,22.253067,25521993.0,235795745.0,2.716443e+06,0.000092
scaffold_4,319504.0,57709.0,32.088281,22.253067,25521993.0,235795745.0,2.716443e+06,0.000012


In [25]:
"""
COMBINING NANOARG FULL TABLES AND ASSEMBLY INFORMATION
------------------------------------------------------

Add scaffold length, read_count and normalized scaffold read_count 
(rpm and rpkm) to NanoARG table.
Store dataframes in a dictionary called all_full_tables_assembly_data.

"""

all_full_tables_assembly_data = {}

for name, keys in all_full_tables.items():    
    df_name = name
    # printing sample name indicates progress
    print(df_name)
    all_full_tables_assembly_data[df_name] = (
        pd.concat([all_full_tables[df_name], assembly_coverage[df_name]], 
                  axis=1, sort=False, join='inner'))

HCL0_W2
HCL0_W3
Before_GAC_1
After_GAC_1


In [26]:
all_full_tables_assembly_data['After_GAC_1'].head()

,gene_id,gene_name,group,category,start,end,strand,identity,bitscore,evalue,...,coverage,is_pathogen,length,read_count,1,all_scaffold,sum_assembled_read,length_sum,coverage_sum,normalized_coverage
scaffold_16508,WP_007306505.1,IS1634 family transposase [Crocosphaera watsonii],MGEs,transposase,1058,1459,+,35.1,102.4,6.500000e-21,...,0.724324,0,1743.0,74.0,6.709122,12.54121,14992547.0,259033383.0,2.552888e+06,0.000003
scaffold_9236,YP_002467958,PBP-1B,ARGs,beta-lactam,326,1816,+,26.6,137.9,1.300000e-32,...,0.718421,0,2392.0,81.0,6.479508,12.54121,14992547.0,259033383.0,2.552888e+06,0.000003
scaffold_9236,UniRef90_Q07259,Cluster: Putative transglycosylase H16 A0665,Functional Genes,Cluster: Putative transglycosylase H16 A0665,395,862,+,37.2,107.8,1.500000e-22,...,0.675325,0,2392.0,81.0,6.479508,12.54121,14992547.0,259033383.0,2.552888e+06,0.000003
scaffold_16501,BAC0162,Gallium,MRGs,Gallium,15,659,+,31.0,102.8,9.400000e-24,...,0.606742,0,1743.0,41.0,4.414802,12.54121,14992547.0,259033383.0,2.552888e+06,0.000002
scaffold_16501,BAC0604,Molybdenum,MRGs,Molybdenum,778,1440,+,30.0,102.8,9.500000e-24,...,0.648256,0,1743.0,41.0,4.414802,12.54121,14992547.0,259033383.0,2.552888e+06,0.000002


In [27]:
"""
SAVE DICTIONARY
---------------
save dictionary in all_full_tables_assembly_data_test.p

dictionary can be loaded in new notebook for analysis

with open('all_full_tables_remove_overlap_test.p', 'rb') as fp:
    all_full_tables_remove_overlap = pickle.load(fp)
"""

# with open('all_full_tables_assembly_data_average_coverage.p', 'wb') as fp:
#     pickle.dump(all_full_tables_assembly_data, fp, protocol=pickle.HIGHEST_PROTOCOL)

"\nSAVE DICTIONARY\n---------------\nsave dictionary in all_full_tables_assembly_data_test.p\n\ndictionary can be loaded in new notebook for analysis\n\nwith open('all_full_tables_remove_overlap_test.p', 'rb') as fp:\n    all_full_tables_remove_overlap = pickle.load(fp)\n"

In [28]:
"""
Running 'remove_overlap_not_ARG_strand_selection'
------------------------------------------------
rearranging with ascending start
and storing updated dataframes in 
a dictionary called
all_full_tables_remove_overlap.

Set the max_overlap to desired value.

IMPORTANT: this step is necessary for running
many of the other functions.

"""
all_full_tables_remove_overlap = {}
for name, keys in all_full_tables_assembly_data.items():    
    df_name = name
    # printing sample name indicates progress
    print(df_name)
    all_full_tables_remove_overlap[df_name] = (
        remove_overlap_not_ARG_strand_selection(all_full_tables_assembly_data[df_name], 
                                            max_overlap=250, 
                                            strand_selection='no',
                                            verbose = False))
       

HCL0_W2
HCL0_W3
Before_GAC_1
After_GAC_1


In [52]:
"""
SAVE DICTIONARY
---------------

Helpful if you have a lot of samples
"""


# with open('all_full_tables_remove_overlap_average_coverage.p', 'wb') as fp:
#     pickle.dump(all_full_tables_remove_overlap, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
assembly['After_GAC_1'].head()

,length,read_count
read,,
scaffold_0,709020,97243
scaffold_1,551495,312878
scaffold_2,954535,669486
scaffold_3,303743,175890
scaffold_4,302455,32823


In [36]:
"""
runs 'count_scaffolds'
---------------------

Returns dataframe with number of scaffolds in a sample.


"""

scaffold_count = pd.DataFrame(dtype=float)
for name, df in assembly_coverage.items(): 
    df_name = name
    # printing sample name indicates progress
    print(df_name)
    count = count_scaffolds(df, min_length = 1000)
    count_series = pd.Series(count)
    count_rename = count_series.rename(name)
    
    scaffold_count = scaffold_count.append(count_rename)

After_GAC_1
Before_GAC_1
HCL0_W2
HCL0_W3


In [37]:
scaffold_count

,# scaffolds
After_GAC_1,46811.0
Before_GAC_1,35063.0
HCL0_W2,40216.0
HCL0_W3,45471.0


In [38]:
"""
runs 'count_scaffolds_with_MGEs_and_ARGs_normalized'
----------------------------------------------------

Necessary to run 'remove_overlap_not_ARG_strand_selection' and 'count_scaffolds' before

Estimate: 5 samples run in ~ 2 mins to run. More samples will take longer. 


Output
------
df : pd.dataframe
    dataframe with counts, ratios and percent values
    
    ARG_counts : Number of ARGs in a sample 
    scaffold_ARGs : Number of scaffolds with at least one ARG 
    scaffold_ARGs_normalized : Scaffold with at least one ARG 
                               rel. abundance 
    scaffold_MGEs : Number of scaffolds with at least one MGE
    scaffold_MGEs_with_ARGs : Number of scaffolds with at least one AGR 
                              and at least one MGE 
    scaffold_MGEs_with_ARGs_normalized : Scaffold with at least one each 
                                         of ARG and MGE rel.abundance
    %_scaffold_MGEs_with_ARGs/scaffold_ARGs : (scaffold_MGEs_with_ARGs x 100)/scaffold_ARGs
    scaffold_pathogens : Number of scaffolds identified as pathogen
    scaffold_pathogens_with_ARGs : Number of scaffolds identified as pathogen with 
                                at least one ARG
    scaffold_pathogens_with_MGEs : Number of scaffolds identified as pathogen with 
                                at least one MGE
    scaffold_pathogens_with_ARGs_MGEs : Number of scaffolds identified 
                                        as pathogen with at least one AGR 
                                        and at least one MGE
    scaffold_has_pathogens_ARGs_MGEs_normalized: Rel abundance of scaffolds identified 
                                        as pathogen with at least one AGR 
                                        and at least one MGE (rpkm)
    # scaffolds :  Number of scaffolds in a sample
    scaffold_MGEs/#scaffolds : scaffold_MGEs/# scaffolds
    %scaffold_ARGs : (scaffold_ARGs  x 100) / # scaffolds
    %scaffold_MGEs_with_ARGs : (scaffold_MGEs_with_ARGs  x 100) / # scaffolds 
    %pathogens : (scaffold_pathogens x 100) / # scaffolds 
    %pathogens_with_ARGs : (scaffold_pathogens_with_ARGs x 100) / # scaffolds
    %pathogens_with_MGEs : (scaffold_pathogens_with_MGEs x 100) / # scaffolds
    %pathogens_with_ARGs_MGEs : (scaffold_pathogens_with_ARGs_MGEs x 100) / # scaffolds 
    %scaffold_MGEs : (scaffold_MGEs x 100) / # scaffolds

"""

columns = [
           'ARG_counts', 'scaffold_ARGs',
           'scaffold_ARGs_normalized',
           'scaffold_MGEs',
           'scaffold_MGEs_with_ARGs',
           'scaffold_MGEs_with_ARGs_normalized',
           '%_scaffold_MGEs_with_ARGs/scaffold_ARGs', 'scaffold_pathogens',
           'scaffold_pathogens_with_ARGs', 'scaffold_pathogens_with_MGEs',
           'scaffold_pathogens_with_ARGs_MGEs',
           'scaffold_has_pathogens_ARGs_MGEs_normalized']
scaffolds_with_MGEs_and_ARGs = pd.DataFrame(columns=columns, dtype=float)
for name, df in all_full_tables_remove_overlap.items(): 
    print(name)
    count = count_scaffolds_with_MGEs_and_ARGs_normalized(df, min_length = 0)
    count_series = pd.Series(count)
    count_rename = count_series.rename(name)
    
    scaffolds_with_MGEs_and_ARGs = scaffolds_with_MGEs_and_ARGs.append(count_rename)
        

scaffolds_with_MGEs_and_ARGs_store = scaffolds_with_MGEs_and_ARGs

MGEs_and_ARGs_pathogens_normalized = (
    pd.concat([scaffolds_with_MGEs_and_ARGs_store, scaffold_count], 
              axis=1, join='inner'))

MGEs_and_ARGs_pathogens_normalized['scaffold_MGEs/#scaffolds']=\
MGEs_and_ARGs_pathogens_normalized['scaffold_MGEs']/\
MGEs_and_ARGs_pathogens_normalized['# scaffolds']

MGEs_and_ARGs_pathogens_normalized['%scaffold_ARGs']=\
MGEs_and_ARGs_pathogens_normalized['scaffold_ARGs']*100/\
MGEs_and_ARGs_pathogens_normalized['# scaffolds']

MGEs_and_ARGs_pathogens_normalized['%scaffold_MGEs_with_ARGs']=\
MGEs_and_ARGs_pathogens_normalized['scaffold_MGEs_with_ARGs']*100/\
MGEs_and_ARGs_pathogens_normalized['# scaffolds']

MGEs_and_ARGs_pathogens_normalized['%pathogens']=\
MGEs_and_ARGs_pathogens_normalized['scaffold_pathogens']*100/\
MGEs_and_ARGs_pathogens_normalized['# scaffolds']

MGEs_and_ARGs_pathogens_normalized['%pathogens_with_ARGs']=\
MGEs_and_ARGs_pathogens_normalized['scaffold_pathogens_with_ARGs']*100/\
MGEs_and_ARGs_pathogens_normalized['# scaffolds']

MGEs_and_ARGs_pathogens_normalized['%pathogens_with_MGEs']=\
MGEs_and_ARGs_pathogens_normalized['scaffold_pathogens_with_MGEs']*100/\
MGEs_and_ARGs_pathogens_normalized['# scaffolds']

MGEs_and_ARGs_pathogens_normalized['%pathogens_with_ARGs_MGEs']=\
MGEs_and_ARGs_pathogens_normalized['scaffold_pathogens_with_ARGs_MGEs']*100/\
MGEs_and_ARGs_pathogens_normalized['# scaffolds']

MGEs_and_ARGs_pathogens_normalized['%scaffold_MGEs']=\
MGEs_and_ARGs_pathogens_normalized['scaffold_MGEs']*100/\
MGEs_and_ARGs_pathogens_normalized['# scaffolds']

HCL0_W2
HCL0_W3
Before_GAC_1
After_GAC_1


In [39]:
"""
Prints result.
Save this file and plot using desired software
"""

MGEs_and_ARGs_pathogens_normalized.columns

Index(['ARG_counts', 'scaffold_ARGs', 'scaffold_ARGs_normalized',
       'scaffold_MGEs', 'scaffold_MGEs_with_ARGs',
       'scaffold_MGEs_with_ARGs_normalized',
       '%_scaffold_MGEs_with_ARGs/scaffold_ARGs', 'scaffold_pathogens',
       'scaffold_pathogens_with_ARGs', 'scaffold_pathogens_with_MGEs',
       'scaffold_pathogens_with_ARGs_MGEs',
       'scaffold_has_pathogens_ARGs_MGEs_normalized', '# scaffolds',
       'scaffold_MGEs/#scaffolds', '%scaffold_ARGs',
       '%scaffold_MGEs_with_ARGs', '%pathogens', '%pathogens_with_ARGs',
       '%pathogens_with_MGEs', '%pathogens_with_ARGs_MGEs', '%scaffold_MGEs'],
      dtype='object')

In [40]:
MGEs_and_ARGs_pathogens_normalized['mean_cov_MGEs_with_ARGs/mean_cov'] = (
(MGEs_and_ARGs_pathogens_normalized['scaffold_MGEs_with_ARGs_normalized']*
    MGEs_and_ARGs_pathogens_normalized['# scaffolds'])/
    MGEs_and_ARGs_pathogens_normalized['scaffold_MGEs_with_ARGs']
)

In [52]:
MGEs_and_ARGs_pathogens_normalized

,ARG_counts,scaffold_ARGs,scaffold_ARGs_normalized,scaffold_MGEs,scaffold_MGEs_with_ARGs,scaffold_MGEs_with_ARGs_normalized,%_scaffold_MGEs_with_ARGs/scaffold_ARGs,scaffold_pathogens,scaffold_pathogens_with_ARGs,scaffold_pathogens_with_MGEs,...,# scaffolds,scaffold_MGEs/#scaffolds,%scaffold_ARGs,%scaffold_MGEs_with_ARGs,%pathogens,%pathogens_with_ARGs,%pathogens_with_MGEs,%pathogens_with_ARGs_MGEs,%scaffold_MGEs,mean_cov_MGEs_with_ARGs/mean_cov
HCL0_W2,6679.0,5248.0,0.037242,3831.0,465.0,0.008308,8.860518,31.0,7.0,22.0,...,40216.0,0.095261,13.049533,1.156256,0.077084,0.017406,0.054705,0.004973,9.526059,0.718541
HCL0_W3,7706.0,6032.0,0.042196,4366.0,550.0,0.009361,9.118037,31.0,5.0,26.0,...,45471.0,0.096017,13.265598,1.209562,0.068175,0.010996,0.057179,0.004398,9.601724,0.773947
Before_GAC_1,4472.0,3766.0,0.019573,2870.0,208.0,0.002843,5.523101,18.0,9.0,11.0,...,35063.0,0.081853,10.740667,0.593218,0.051336,0.025668,0.031372,0.005704,8.185267,0.479255
After_GAC_1,5358.0,4264.0,0.017275,3561.0,390.0,0.003449,9.146341,3.0,1.0,2.0,...,46811.0,0.076072,9.108970,0.833138,0.006409,0.002136,0.004273,0.000000,7.607186,0.414036


In [41]:
"""
runs 'ARG_subtype_count'
-----------------------


Run 'scaffold_count' before with desired scaffold length threshhold.

Returns a dataframe with count of ARG subtype.
"""

columns = ['arg_subtype_count', 'scaffold_arg_subtype', 
           'scaffold_has_arg_subtype_normalized']
ARG_subtypes = pd.DataFrame(columns=columns, dtype=float)
for name, df in all_full_tables_assembly_data.items():    
    count = ARG_subtype_count(df, min_bitscore=50, 
                      min_id_arg=50, min_coverage_arg = 0.5, arg_name = 'msrE')
    count_series = pd.Series(count)
    count_rename = count_series.rename(name)
    
    ARG_subtypes = ARG_subtypes.append(count_rename)
        

ARG_subtypes_store = ARG_subtypes 


ARG_subtypes_scaffold_count = \
pd.concat([ARG_subtypes_store, scaffold_count], axis=1, join='inner')

ARG_subtypes_scaffold_count['%scaffold_arg_subtype']=\
ARG_subtypes_scaffold_count['scaffold_arg_subtype']*100/\
ARG_subtypes_scaffold_count['# scaffolds']

In [42]:
ARG_subtypes_scaffold_count

,arg_subtype_count,scaffold_arg_subtype,scaffold_has_arg_subtype_normalized,# scaffolds,%scaffold_arg_subtype
HCL0_W2,1.0,1.0,0.000010,40216.0,0.002487
HCL0_W3,1.0,1.0,0.000019,45471.0,0.002199
Before_GAC_1,1.0,1.0,0.000005,35063.0,0.002852
After_GAC_1,1.0,1.0,0.000003,46811.0,0.002136


In [45]:
pd.set_option('display.max_rows', 100)

In [43]:
"""
Step necessary before merging samples
-------------------------------------

Adds sample names (exp. 'HCL0_W2') to 
scaffold numbers in index (exp. scaffold_0).
The modified index will have both sample name 
and scaffold number.
Dataframes will be updated with the new combined index
and stored in the same dictionary called
all_full_tables_remove_overlap.

"""

for name, df in all_full_tables_remove_overlap.items():
    df['file_name'] = name
    df['index'] = df.index
    df['read'] = df['file_name'] + df['index']
    df.index = df['read']
    df = df.drop(['read', 'file_name', 'index'], axis=1 )

In [44]:
"""
MERGING SAMPLES TO DESIRED GROUPS
---------------------------------


Combine samples to be analyzed as a group
and store as pd.DataFrame.

For example,
HCL0_W2 and HCL0_W3 are two NanoARG tables, 
chlorine_unfiltered is the combined table.

"""


chlorine_unfiltered = pd.concat([
                                all_full_tables_remove_overlap['HCL0_W2'],
                                all_full_tables_remove_overlap['HCL0_W3']
                                ])

In [48]:
"""
Running 'ARG_MGE_association'
-----------------------------


Change 'chlorine_unfiltered' to your sample name.
Change parameters as desired. Parameter not specified runs with default.

Set the ARG aa idendity higher and lower, change max_fdr etc. to explore the data.
For example, setting high identity and coverage can return ARGs 
known to be associated with mobile genetic elements.

Import/deduce 'f' from dataframe 'MGEs_and_ARGs_pathogens_normalized'. 

Group samples differently for data exploration. 

"""


chlorine_unfiltered_ARG_MGE_association = ARG_MGE_association(
    chlorine_unfiltered, min_id_arg=50, 
    f = 0.01,
    min_coverage_arg=0.4)

In [50]:
"""
Running 'neighbor_stretch'
--------------------------
Change 'chlorine_unfiltered' to your sample name.
Change parameters as desired. 
For example, setting max_distance higher looks at a longer stretch of the scaffold.

"""


neighbor_stretch_unfiltered_chlorine = neighbor_stretch(
    chlorine_unfiltered, min_ARG_count=1, max_distance =5000, 
                     number_neighboring_orf=4, 
                     sliding_window_step_increase=4, min_bitscore=50, 
                     min_id_arg=80, min_id_mge=25, min_coverage_arg=0.5, verbose=False)

In [51]:
neighbor_stretch_unfiltered_chlorine

,gene_id,gene_name,group,category,start,end,strand,identity,bitscore,evalue,NCBI_taxa_id,taxa_centrifuge_score,species,coverage,is_pathogen
read,,,,,,,,,,,,,,,
HCL0_W2scaffold_3375,ANP63073.1,mphD,ARGs,MLS,4070,4882,+,100.0,548.1,1.600000e-155,48296,302771.0,Acinetobacter pittii,0.921769,0
HCL0_W2scaffold_3375,YP_724476.1,msrE,ARGs,MLS,4941,6413,+,100.0,943.0,2.300000e-274,48296,302771.0,Acinetobacter pittii,1.000000,0
HCL0_W2scaffold_3375,UniRef90_P55373,Cluster: Putative transposase y4bF,Functional Genes,Cluster: Putative transposase y4bF,6968,8221,+,46.9,376.3,9.200000e-103,48296,302771.0,Acinetobacter pittii,0.923414,0
HCL0_W2scaffold_3375,WP_067143567.1,transposase [Oceanivirga salmonicida],MGEs,transposase,7331,8017,+,30.9,100.1,1.800000e-19,48296,302771.0,Acinetobacter pittii,0.682493,0
